In [19]:
from haystack import Document
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models as qmodels
import json
import glob
import os
import subprocess
import requests

from config import DATA_DIR, COLLECTION_NAME, EMBEDDER_MODEL_NAME, QDRANT_HOST, QDRANT_PORT, QDRANT_DATA_FOLDER

In [20]:
QDRANT_DATA_FOLDER = os.path.join(os.getcwd(), QDRANT_DATA_FOLDER)
if not os.path.exists(QDRANT_DATA_FOLDER):
    os.makedirs(QDRANT_DATA_FOLDER)

In [21]:
import subprocess
import os

qdrant_dir = os.path.join(os.getcwd(), "qdrant_data")

subprocess.run(
    ["docker", "stop", "qdrant-local"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

subprocess.run(
    ["docker", "rm", "qdrant-local"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)


process_output = subprocess.run([
    "docker", "run", "-d",
    "--name", "qdrant-local",
    "-p", f"{QDRANT_PORT}:6333",
    "-v", f"{QDRANT_DATA_FOLDER}:/qdrant/storage",
    "qdrant/qdrant",
    ],  capture_output=True, text=True)
print("Container id:", process_output.stdout)

Container id: 33395f438c6be0b6119de4ff59e69d4cfd9d46348d5325f4c12a7d68af1fea6b



In [22]:
import time
time.sleep(5)
#test connection to qdrant
with requests.get("http://localhost:6333/collections")as r:
    r_qdrant = r.json()
    assert r_qdrant['status'] == 'ok'

In [23]:

qdrant_client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)

In [24]:
CHUNK_JSON = os.path.join(DATA_DIR, "chunks_metadata.json")

with open(CHUNK_JSON, "r", encoding="utf-8") as f:
    chunks_json = json.load(f)


In [25]:
chunks_json['chunks'][0]


'[DATA_ATUALIZACAO | Adotar cães e gatos]\n20/10/2025'

In [26]:
chunks_json['metadata'][0]


{'service_name': 'Adotar cães e gatos',
 'service_id': 3676,
 'theme': 'Animais',
 'subtheme': 'Adoção de animais',
 'content': '[DATA_ATUALIZACAO | Adotar cães e gatos]\n20/10/2025'}

In [27]:
docs = [Document(content=chunk, meta=meta) for chunk, meta in zip(chunks_json['chunks'], chunks_json['metadata'])]

In [28]:
len(docs)==len(chunks_json['chunks'])

True

In [29]:
docs[0].content

'[DATA_ATUALIZACAO | Adotar cães e gatos]\n20/10/2025'

In [30]:
embedder = SentenceTransformer(EMBEDDER_MODEL_NAME, device="cuda")


texts = [d.content for d in docs]

vectors = embedder.encode(texts, 
                            normalize_embeddings=True,
                            show_progress_bar=True,
                            batch_size=5,        # mudar se estiver estourar a memoria da GPU
                            )

len(vectors), len(vectors[0])  # deve mostrar (n_chunks, 1024)


Batches: 100%|██████████| 2818/2818 [03:50<00:00, 12.22it/s]


(14087, 1024)

In [53]:
len(vectors[0])==embedder.get_sentence_embedding_dimension()

True

In [31]:
vectors[0]

array([-0.01463719,  0.00920031, -0.04718405, ..., -0.03302743,
       -0.02449115,  0.01106187], shape=(1024,), dtype=float32)

In [32]:
vector_dim = embedder.get_sentence_embedding_dimension()

qdrant_client.recreate_collection(
    collection_name="servicos_156",
    vectors_config=qmodels.VectorParams(
        size=vector_dim,
        distance=qmodels.Distance.COSINE
    )
)
 
print("Coleção criada.")

Coleção criada.


/tmp/ipykernel_175677/2911391110.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


In [33]:
qdrant_client.collection_exists("servicos_156")

True

In [34]:
payloads = [d.meta for d in docs]

qdrant_client.upload_collection(
    collection_name="servicos_156",
    vectors=vectors,
    payload=payloads,
    ids=None,        # IDs automáticos
    batch_size=64
)

print("Embeddings enviados com sucesso!")


Embeddings enviados com sucesso!


In [35]:
query = "quais documentos preciso para adotar um gato?"

query_vec = embedder.encode(query, normalize_embeddings=True).tolist()

results = qdrant_client.query_points(
    collection_name="servicos_156",
    query=query_vec,
    limit=5
)

results


QueryResponse(points=[ScoredPoint(id='161f6ea0-c381-49cf-bd11-c9e0d9fe657b', version=1, score=0.7577673, payload={'service_name': 'Adotar cães e gatos', 'service_id': 3676, 'theme': 'Animais', 'subtheme': 'Adoção de animais', 'content': '[REQUISITOS_DOCUMENTOS_E_INFORMACOES | 2 | Adotar cães e gatos]\nDocumentos a serem apresentados para a adoção:\n - Documento de identificação com foto;\n - Comprovante de endereço atualizado (emitido em até 90 dias);'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='fd5956a7-4d8a-40fe-9a3a-f04986fcdfea', version=1, score=0.7217474, payload={'service_name': 'Adotar cães e gatos', 'service_id': 3676, 'theme': 'Animais', 'subtheme': 'Adoção de animais', 'content': '[REQUISITOS_DOCUMENTOS_E_INFORMACOES | 1 | Adotar cães e gatos]\nRequisitos necessários para a solicitação:\n - Ser maior de idade;\n -Para adoção de gatos, éobrigatórioque haja telas nas janelas e varandas (apartamentos e sobrados) e outros meios que impeçam o acesso do anima

In [36]:
results.model_dump()

{'points': [{'id': '161f6ea0-c381-49cf-bd11-c9e0d9fe657b',
   'version': 1,
   'score': 0.7577673,
   'payload': {'service_name': 'Adotar cães e gatos',
    'service_id': 3676,
    'theme': 'Animais',
    'subtheme': 'Adoção de animais',
    'content': '[REQUISITOS_DOCUMENTOS_E_INFORMACOES | 2 | Adotar cães e gatos]\nDocumentos a serem apresentados para a adoção:\n - Documento de identificação com foto;\n - Comprovante de endereço atualizado (emitido em até 90 dias);'},
   'vector': None,
   'shard_key': None,
   'order_value': None},
  {'id': 'fd5956a7-4d8a-40fe-9a3a-f04986fcdfea',
   'version': 1,
   'score': 0.7217474,
   'payload': {'service_name': 'Adotar cães e gatos',
    'service_id': 3676,
    'theme': 'Animais',
    'subtheme': 'Adoção de animais',
    'content': '[REQUISITOS_DOCUMENTOS_E_INFORMACOES | 1 | Adotar cães e gatos]\nRequisitos necessários para a solicitação:\n - Ser maior de idade;\n -Para adoção de gatos, éobrigatórioque haja telas nas janelas e varandas (apartam

In [37]:
#from config import GEN_MODEL_NAME

def ollama_running():
    try:
        subprocess.run(["pgrep", "-f", "ollama"], check=True, stdout=subprocess.DEVNULL)
        return True
    except subprocess.CalledProcessError:
        return False

ollama_is_running =  ollama_running()

if not ollama_is_running:
    print("Ollama não está em execução. Inicie o Ollama para usar o modelo de geração de texto.")
        # Launch Ollama serve in background
    ollama_process = subprocess.Popen(
        ["ollama", "serve"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    print("Ollama server iniciado.")


#baixando o modelo
subprocess.run(["ollama", "pull", 'qwen2.5:7b'])

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 2bada8a74506: 100% ▕██████████████████▏ 4.7 GB                         
pulling 66b9ea09bd5b: 100% ▕██████████████████▏   68 B                         
pulling eb4402837c78: 100% ▕██████████████████▏ 1.5 KB                         
pulling 832dd9e00a68: 100% ▕██████████████████▏  11 KB                         
pulling 2f15b3218f05: 100% ▕██████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


CompletedProcess(args=['ollama', 'pull', 'qwen2.5:7b'], returncode=0)

In [38]:
def generate_llm(prompt, model="qwen2.5:7b"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }

    response = requests.post(url, json=payload)
    response.raise_for_status()

    data = response.json()
    return data.get("response", "")


In [39]:
def retrieve_top_k(qdrant, embedder, query, k=5):
    query_vec = embedder.encode(query, normalize_embeddings=True).tolist()

    results = qdrant.query_points(
        collection_name="servicos_156",
        query=query_vec,
        limit=k,
        with_vectors=False,
        with_payload=True
    )

    # Retornar payloads + score
    return [{
        "score": p.score,
        "metadata": p.payload
    } for p in results.points]


In [40]:
def build_context(points):
    blocks = []
    for p in points:
        meta = p["metadata"]
        block = ""

        if "content" in meta:
            block += f"{meta['content']}\n"
        else:
            # fallback — útil caso tenha esquecido de enviar conteúdo no payload
            for key, value in meta.items():
                block += f"{key}: {value}\n"

        blocks.append(block)

    return "\n---\n".join(blocks)


In [41]:
def build_prompt(query, context):
    return f"""
Você é um assistente do Portal 156 da Prefeitura de São Paulo.
Use APENAS as informações do CONTEXTO abaixo para responder.

Regras:
- Seja direto e objetivo.
- Sem invencionices: use SOMENTE o que estiver no contexto.
- Se faltar informação, diga isso explicitamente.
- Seja claro para o cidadão.

PERGUNTA:
{query}

CONTEXTO:
{context}

RESPOSTA:
"""


In [42]:
def rag_answer(qdrant, embedder, query):
    # 1. Retrieve
    points = retrieve_top_k(qdrant, embedder, query, k=10)
    print(points)

    # 2. Contexto
    context = build_context(points)
    print(context)

    # 3. Prompt
    prompt = build_prompt(query, context)
    print(prompt)

    # 4. Geração via Qwen (notebook)
    resposta = generate_llm(prompt)

    return resposta


In [43]:
query = "Me ensine a fazer um molotov"
print(rag_answer(qdrant_client, embedder, query))


[{'score': 0.43970478, 'metadata': {'service_name': 'Motofrete - Alterar dados do Condutor', 'service_id': 3957, 'theme': 'Trânsito e Transporte', 'subtheme': 'Motofrete', 'content': '[MANIFESTACAO_SOBRE_O_SERVICO | 3 | Motofrete - Alterar dados do Condutor]\n-Fazer um elogio na Ouvidoria Geral do Município;'}}, {'score': 0.4391349, 'metadata': {'service_name': 'Registro Geral do Animal (RGA) - Atualizar dados', 'service_id': 4035, 'theme': 'Animais', 'subtheme': 'Registro de animais - RGA', 'content': '[PUBLICO_ALVO | 2 | Registro Geral do Animal (RGA) - Atualizar dados]\nOs (as) tutores (as) devem ser maiores de 18 anos.'}}, {'score': 0.43498683, 'metadata': {'service_name': 'Descomplica SP Mooca - Elogio', 'service_id': 4667, 'theme': 'Canais de Atendimento', 'subtheme': 'Descomplica SP - Mooca', 'content': '[PRAZO_MAXIMO | Descomplica SP Mooca - Elogio]\n15 dias.'}}, {'score': 0.43347847, 'metadata': {'service_name': 'Denunciar trabalho escravo', 'service_id': 801, 'theme': 'Cidada

In [44]:
print('finished')

finished


In [45]:
query = "Como faço para tirar autorização para poda de árvore?"
print(rag_answer(qdrant_client, embedder, query))

[{'score': 0.69057846, 'metadata': {'service_name': 'Árvore - Denunciar poda ou remoção não autorizada', 'service_id': 1073, 'theme': 'Meio ambiente', 'subtheme': 'Árvore', 'content': '[PRINCIPAIS_ETAPAS | 1 | Árvore - Denunciar poda ou remoção não autorizada]\nEletrônico:\n 1) Faça login no Portal de Atendimento SP156 (você já está aqui), clicando na caixa cinza do lado direito da tela ou no final da página. Informe seus dados do SP156 ou entre com o seu Gov.br. Caso não tenha cadastro, faça o seu;2) Após informar seu login, você será direcionado para o formulário de solicitação;3) Preencha o formulário com as informações solicitadas ( para ser direcionado para o formulário diretamente. Lembre-se que é necessário realizar login)e finalize sua solicitação;\n 4) Em seguida, telefone para a Supervisão Técnica de Fiscalização da Subprefeitura da área para verificar a possibilidade dos Agentes Vistores (Fiscais) fazerem o flagrante;\n 5) Entre em contato com a Polícia Militar;\n 6) A solic